<a href="https://colab.research.google.com/github/vishnu-chand/moneyTalks/blob/master/moneyTalks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lets make the "Money Talks"
## Implementation of harry potter magic using deep learning :P

Courtesy:  
I used modules from, 
*   [face motion](https://github.com/AliaksandrSiarohin/first-order-model)
*   [face detection](https://github.com/the-house-of-black-and-white/hall-of-faces.git)
*   [face overlay](https://www.learnopencv.com/seamless-cloning-using-opencv-python-cpp/)
*   [opencv](https://opencv.org/)



**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

![alt text](https://raw.githubusercontent.com/vishnu-chand/moneyTalks/master/addFolder.png)

In [0]:
# Clone repositories
%matplotlib inline
!git clone https://github.com/vishnu-chand/moneyTalks.git
!git clone https://github.com/AliaksandrSiarohin/first-order-model
!pip install -q -U imutils git+https://github.com/the-house-of-black-and-white/hall-of-faces.git

Cloning into 'moneyTalks'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.
Cloning into 'first-order-model'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 212 (delta 5), reused 11 (delta 3), pack-reused 197
Receiving objects: 100% (212/212), 71.45 MiB | 38.73 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [0]:
cd first-order-model

/content/first-order-model


In [0]:
import cv2
import warnings
import numpy as np
from glob import glob
import ipywidgets as widgets
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
import matplotlib.pyplot as plt
from ipywidgets import interact
from demo import make_animation
from skimage import img_as_ubyte
from IPython.display import HTML
warnings.filterwarnings("ignore")
from IPython.display import Image
from demo import load_checkpoints
from IPython.display import display
from skimage.transform import resize
from PIL.Image import fromarray as Img
import matplotlib.animation as animation
from IPython.display import clear_output
from hof.face_detectors import YOLOv2FaceDetector

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Create and load model
MIN_CONFIDENCE=0.5
yolo_face_detector = YOLOv2FaceDetector(min_confidence=MIN_CONFIDENCE)

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

In [0]:
print("upload images and videos")
files.upload()

imExts = ['bmp', 'jpg', 'jpeg',  'png']
vdExts = ['mp4', 'avi']
bookPaths = glob('../moneyTalks/*.*') + glob('*.*')
imPaths, vdPaths = list(), list()
for bookPath in bookPaths:
  for ext in  imExts:
    if bookPath.endswith(ext):
      imPaths.append(bookPath)
      break

for bookPath in bookPaths:
  for ext in vdExts:
    if bookPath.endswith(ext):
      vdPaths.append(bookPath)
      break

print("imPaths:", imPaths)
print("vdPaths:", vdPaths)

upload images and videos


Saving poda.mp4 to poda.mp4
imPaths: ['../moneyTalks/dollar.jpg', '../moneyTalks/dime.jpg', '../moneyTalks/ausDollar.jpeg', '../moneyTalks/addFolder.png']
vdPaths: ['../moneyTalks/trump.mp4', 'poda.mp4', 'out.avi']


In [0]:
input_image = None
def chooseImg(imPath):
  global input_image
  clear_output()
  input_image = cv2.imread(imPath)[:,:,::-1].copy()
  display(Img(input_image))
interact(chooseImg, imPath=imPaths)

vdPath, input_video = None, None
def chooseVideo(_vdPath):
  clear_output()
  global vdPath, input_video
  cam, vdPath = cv2.VideoCapture(_vdPath), _vdPath
  input_video = cam.read()[1][:, :, ::-1].copy()
  display(Img(input_video))
interact(chooseVideo, _vdPath=vdPaths)

interactive(children=(Dropdown(description='imPath', options=('../moneyTalks/dollar.jpg', '../moneyTalks/dime.…

interactive(children=(Dropdown(description='_vdPath', options=('../moneyTalks/trump.mp4', 'poda.mp4', 'out.avi…

<function __main__.chooseVideo>

In [0]:
imH, imW = input_image.shape[:2]
faces = yolo_face_detector.detect(input_image.copy(), draw_faces=False)
x,y,w,h = max(faces, key=lambda r: r[-1]*r[-2])
pad = .4 
pad = int(pad * w), int(pad * h)
x,y,w,h = x - pad[0], y - pad[1], w + 2 * pad[0], h + 2 * pad[1]

def markFace(left, right, top, bottom):
  global x0,y0, x1, y1
  clear_output()
  img = input_image.copy()
  x0, y0, x1, y1 = x, y, x + w, y + h
  x0, y0, x1, y1 = x0 + left, y0 + top, x1 + right, y1 + bottom
  x0, y0, x1, y1 = max(0, x0), max(0, y0), min(imW, x1), min(imH, y1)
  cv2.rectangle(img, (x0,y0), (x1,y1), color=255, thickness=5) 
  display(Img(img))
v = int(.2 * max(w, h))
v = (-v,v,10)
interact(markFace, left=v, right=v, top=v, bottom=v)

interactive(children=(IntSlider(value=-6, description='left', max=26, min=-26, step=10), IntSlider(value=-6, d…

<function __main__.markFace>

In [0]:
imH, imW = input_video.shape[:2]
faces = yolo_face_detector.detect(input_video.copy(), draw_faces=False)
x,y,w,h = max(faces, key=lambda r: r[-1]*r[-2])
pad = .4 
pad = int(pad * w), int(pad * h)
x,y,w,h = x - pad[0], y - pad[1], w + 2 * pad[0], h + 2 * pad[1]

def markFace(left, right, top, bottom):
  global vx0,vy0, vx1, vy1 
  img = input_video.copy()
  vx0, vy0, vx1, vy1 = x, y, x + w, y + h
  vx0, vy0, vx1, vy1 = vx0 + left, vy0 + top, vx1 + right, vy1 + bottom
  vx0, vy0, vx1, vy1 = max(0, vx0), max(0, vy0), min(imW, vx1), min(imH, vy1)
  cv2.rectangle(img, (vx0,vy0), (vx1,vy1), color=255, thickness=5) 
  display(Img(img))
v = int(.2 * max(w, h))
v = (-v,v,10)
interact(markFace, left=v, right=v, top=v, bottom=v)

interactive(children=(IntSlider(value=-1, description='left', max=61, min=-61, step=10), IntSlider(value=-1, d…

<function __main__.markFace>

In [0]:
#Resize image and video to 256x256
source_image = cv2.resize(input_image[y0:y1, x0:x1], (256, 256))
cam = cv2.VideoCapture(vdPath)
puppet_video = []
while True:
  _, frame = cam.read()
  if frame is None:
    break
  h, w = frame.shape[:2]
  frame = frame[vy0:vy1, vx0:vx1][:, :, ::-1].copy()
  frame = cv2.resize(frame, (256, 256))
  puppet_video.append(frame)

def displayVideo(puppet_video, generated, show_puppet, figSize=96):
    for i in range(len(puppet_video)):
        img = generated[i] if type(generated) == list else generated
        if show_puppet:
          i1 = puppet_video[i]
          (h1, w1), (h2, w2) = i1.shape[:2], img.shape[:2]
          i1 = cv2.resize(i1, (int(h1/w1) * h2, h2))
          img = cv2.hconcat([i1,img])
        if i == 0:
            h, w = img.shape[:2]
            fig = plt.figure(figsize=(w/figSize, h/figSize)) # assume 128pix == 1 inch
            ims = []
        im = plt.imshow(img, animated=True)
        plt.axis('off')
        plt.tight_layout()
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
HTML(displayVideo(puppet_video, source_image, True).to_html5_video())

In [0]:
# animate
predictions = make_animation(source_image/255.0, np.array(puppet_video)/255.0, generator, kp_detector, relative=True)
for ix, outImg in enumerate(predictions):
  outImg = (outImg * 255).astype('u1')
  outImg = cv2.resize(outImg, (x1 - x0, y1 - y0), interpolation=cv2.INTER_CUBIC)
  img = input_image.copy()
  img[y0:y1, x0:x1] = outImg
  m = np.zeros_like(img)
  m[y0:y1, x0:x1] = 255
  c = int((x0 + x1) / 2), int((y0 + y1) / 2)
  img = cv2.seamlessClone(img, input_image, m, c, cv2.NORMAL_CLONE)
  predictions[ix] = img # update prediction
HTML(displayVideo(puppet_video, predictions, True).to_html5_video())

100%|██████████| 211/211 [00:26<00:00,  7.99it/s]
